In [3]:
import pandas as pd
import spacy
from collections import defaultdict
from textblob import TextBlob

In [5]:
df=pd.read_csv('flipkart_sent.csv')
print(df.shape)
df=df.head(5000)
df.head()


(25000, 6)


,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [31]:
nlp = spacy.load("en_core_web_sm")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [32]:
def extract_aspects_and_sentiment(text):
    doc = nlp(text)
    aspects_sentiment = []
    
    for chunk in doc.noun_chunks:
        aspect = chunk.text.lower()
        sentiment = TextBlob(chunk.sent.text).sentiment.polarity
        aspects_sentiment.append((aspect, sentiment))
        
    return aspects_sentiment

In [15]:
from tqdm.notebook import tqdm  # Use `notebook` for Jupyter compatibility
from collections import defaultdict

product_aspects = defaultdict(list)

# Wrap df.iterrows() with tqdm
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing reviews"):
    product = row['product_name']
    review = str(row['Review']) + " " + str(row['Summary'])
    aspects = extract_aspects_and_sentiment(review)
    
    for aspect, sentiment in aspects:
        product_aspects[product].append((aspect, sentiment))

Processing reviews:   0%|          | 0/25000 [00:00<?, ?it/s]

In [18]:
product_summary = []

for product, aspects in product_aspects.items():
    aspect_stats = defaultdict(list)
    for aspect, sentiment in aspects:
        aspect_stats[aspect].append(sentiment)
    
    top_aspects = sorted(aspect_stats.items(), key=lambda x: -len(x[1]))[:5]  # Top 5 aspects
    summary = {f"{asp}_sentiment": sum(sents)/len(sents) for asp, sents in top_aspects}
    summary["product_name"] = product
    product_summary.append(summary)

final_df = pd.DataFrame(product_summary)

print(final_df.head())
final_df.to_csv("flipkart_aspect.csv", index=False)
# Save the final DataFrame to a CSV file


   super! great cooler excellent air flow_sentiment  this price_sentiment  \
0                                          0.663333              0.663333   
1                                               NaN                   NaN   
2                                               NaN                   NaN   
3                                               NaN                   NaN   
4                                               NaN                   NaN   

   unbelievablejust_sentiment  it_sentiment  awesome best budget_sentiment  \
0                    0.663333      0.663333                           0.75   
1                         NaN           NaN                            NaN   
2                         NaN      0.299250                            NaN   
3                         NaN      0.274928                            NaN   
4                         NaN      0.369690                            NaN   

                                        product_name  nice product_s

In [19]:
import pandas as pd
df=pd.read_csv('flipkart_aspect.csv')
print(df.shape)
df=df.head(5000)
df.head()


(131, 148)


,super! great cooler excellent air flow_sentiment,this price_sentiment,unbelievablejust_sentiment,it_sentiment,awesome best budget_sentiment,product_name,nice product_sentiment,great product beautiful product good material_sentiment,mind-blowing purchase awesome_sentiment,brilliant wonderful product_sentiment,...,who_sentiment,timejars_sentiment,box_sentiment,itself_sentiment,the surface_sentiment,your money_sentiment,the job soo_sentiment,music_sentiment,google_sentiment,india_sentiment
0,0.663333,0.663333,0.663333,0.663333,0.75,Candes 12 L Room/Personal Air Cooler??????(Whi...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Candes 60 L Room/Personal Air Cooler??????(Whi...,0.5,0.8375,1.0,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0.299250,NaN,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.274928,NaN,"Crompton 75 L Desert Air Cooler??????(White, T...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.369690,NaN,boAt Rockerz 510 Super Extra Bass Bluetooth He...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
from tqdm.notebook import tqdm  # Use `notebook` for Jupyter compatibility
from collections import defaultdict

aspect_df = defaultdict(list)

import pandas as pd
import spacy
from collections import defaultdict

# Load dataset
df = pd.read_csv("flipkart_sent.csv")

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Positive and negative keyword lists
positive_words = {'good', 'great', 'excellent', 'awesome', 'amazing', 'perfect', 'fast', 'cheap', 'durable'}
negative_words = {'bad', 'poor', 'worst', 'slow', 'expensive', 'terrible', 'useless', 'cheap quality'}

# Combine review and summary for better context
df["full_review"] = df["Review"].fillna('') + " " + df["Summary"].fillna('')

# Store final aspect sentiment summaries
product_aspect_summary = []

# Process each product
for product_name in df['product_name'].unique():
    product_df = df[df['product_name'] == product_name].head(100)  # Limit to 100 reviews for memory safety
    aspect_sentiments = defaultdict(list)
    
    for review in product_df["full_review"]:
        doc = nlp(review)
        for chunk in doc.noun_chunks:
            aspect = chunk.text.lower().strip()
            sent_score = 0
            tokens = [t.text.lower() for t in chunk.sent]
            for word in tokens:
                if word in positive_words:
                    sent_score += 1
                elif word in negative_words:
                    sent_score -= 1
            if sent_score != 0:
                aspect_sentiments[aspect].append(sent_score)

    # Take top 5 aspects by frequency
    aspect_data = sorted(aspect_sentiments.items(), key=lambda x: -len(x[1]))[:5]
    summary = {f"{asp}_sentiment": round(sum(scores)/len(scores), 2) for asp, scores in aspect_data}
    summary["product_name"] = product_name
    product_aspect_summary.append(summary)

# Convert result to DataFrame
aspect_df = pd.DataFrame(product_aspect_summary)

# Display the first few rows
print(aspect_df.head())

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


   super! great cooler excellent air flow_sentiment  this price_sentiment  \
0                                               3.0                   3.0   
1                                               NaN                   NaN   
2                                               NaN                   NaN   
3                                               NaN                   NaN   
4                                               NaN                   NaN   

   unbelievablejust_sentiment  it_sentiment  awesome best budget_sentiment  \
0                         3.0          3.00                            1.0   
1                         NaN           NaN                            NaN   
2                         NaN          1.69                            NaN   
3                         NaN          1.75                            NaN   
4                         NaN          1.96                            NaN   

                                        product_name  \
0  Candes 12

In [8]:
df=pd.read_csv('flipkart_aspect.csv')
print(df.shape)
df=df.head(5000)
df.head()


(131, 148)


,super! great cooler excellent air flow_sentiment,this price_sentiment,unbelievablejust_sentiment,it_sentiment,awesome best budget_sentiment,product_name,nice product_sentiment,great product beautiful product good material_sentiment,mind-blowing purchase awesome_sentiment,brilliant wonderful product_sentiment,...,who_sentiment,timejars_sentiment,box_sentiment,itself_sentiment,the surface_sentiment,your money_sentiment,the job soo_sentiment,music_sentiment,google_sentiment,india_sentiment
0,0.663333,0.663333,0.663333,0.663333,0.75,Candes 12 L Room/Personal Air Cooler??????(Whi...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Candes 60 L Room/Personal Air Cooler??????(Whi...,0.5,0.8375,1.0,0.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,0.299250,NaN,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.274928,NaN,"Crompton 75 L Desert Air Cooler??????(White, T...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.369690,NaN,boAt Rockerz 510 Super Extra Bass Bluetooth He...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
